#Initialisation

##Imports

In [1]:
!pip install ktrain

     |████████████████████████████████| 25.3 MB 64 kB/s 
     |████████████████████████████████| 6.8 MB 17.8 MB/s 
     |████████████████████████████████| 981 kB 51.6 MB/s 
     |████████████████████████████████| 263 kB 54.6 MB/s 
     |████████████████████████████████| 2.8 MB 43.7 MB/s 
     |████████████████████████████████| 1.2 MB 46.7 MB/s 
     |████████████████████████████████| 468 kB 54.3 MB/s 
     |████████████████████████████████| 3.3 MB 41.5 MB/s 
     |████████████████████████████████| 895 kB 50.9 MB/s 
     |████████████████████████████████| 596 kB 66.6 MB/s 
     |████████████████████████████████| 56 kB 4.6 MB/s 
  Created wheel for ktrain: filename=ktrain-0.28.2-py3-none-any.whl size=25290606 sha256=34b59618bcc5fe1311281d5a2f5c4fadba07e8336e250c0808143deb05600c0f
  Stored in directory: /root/.cache/pip/wheels/df/2b/04/7c821b51e637ec480060989b5030d0c4cce16efe0d67bff94b
  Created wheel for seqeval: filename=seqeval-0.0.19-py3-none-any.whl size=9929 sha256=fea47e72fb6c65e31

In [2]:
import re
import json
import pandas as pd
import numpy as np
import spacy
import statistics
import ktrain
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from ktrain import text

from numpy import nan
import matplotlib.pyplot as plt

%reload_ext autoreload
%autoreload 2
%matplotlib inline
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"]="0"; 

##Mountdrive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Set Gpu

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 1GB of memory on the first GPU
  try:
    tf.config.set_logical_device_configuration(
        gpus[0],
        [tf.config.LogicalDeviceConfiguration(memory_limit=1024)])
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

#Load data

In [4]:
#DATA = pd.read_csv('/content/drive/MyDrive/DATA_full.csv')
MBTI = pd.read_csv('/content/drive/MyDrive/MBTI_full.csv')

#DATA = DATA.drop(['Unnamed: 0', 'Segment'], axis='columns')
MBTI = MBTI.drop(['Unnamed: 0', 'Segment'], axis='columns')

#Text classification

###setup

In [5]:
df = MBTI[['posts','type']]
df.reset_index(drop=True, inplace=True)
df = pd.concat([df, df.type.astype('str').str.get_dummies()], axis=1, sort=False)
df.drop('type', axis=1, inplace=True)
df.rename(columns={'posts':'text'}, inplace=True)

In [6]:
df.head()

,text,ENFJ,ENFP,ENTJ,ENTP,ESFJ,ESFP,ESTJ,ESTP,INFJ,INFP,INTJ,INTP,ISFJ,ISFP,ISTJ,ISTP
0,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,'I'm finding the lack of me in these posts ver...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,'Good one _____ https://www.youtube.com/wat...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,"'Dear INTP, I enjoyed our conversation the o...",0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,'You're fired.|||That's another silly misconce...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [7]:
Labels_names = df.columns.values[1:]

In [8]:
LIWCs = MBTI.loc[:, ~MBTI.columns.isin(['type', 'posts'])]

In [9]:
train_size = int(len(MBTI) * .8)
print ("Train size: %d" % train_size)
print ("Test size: %d" % (len(MBTI) - train_size))

Train size: 6940
Test size: 1735


In [11]:
labels = df.loc[:, df.columns!='text']

In [12]:
# Train features
description_train = df['text'][:train_size]
LIWCs_train = LIWCs[:train_size]
# Train labels
labels_train = labels[:train_size]

# Test features
description_test = df['text'][train_size:]
LIWCs_test = LIWCs[train_size:]
# Test labels
labels_test = labels[train_size:]

x_train = description_train.values
y_train = labels_train.values
LIWCs_train = LIWCs_train.values
x_test = description_test.values
y_test = labels_test.values
LIWCs_test = LIWCs_test.values

In [ ]:
'''# load text data
categories = ['alt.atheism', 'soc.religion.christian','comp.graphics', 'sci.med']
from sklearn.datasets import fetch_20newsgroups
train_b = fetch_20newsgroups(subset='train', categories=categories, shuffle=True)
test_b = fetch_20newsgroups(subset='test',categories=categories, shuffle=True)
(x_train, y_train) = (train_b.data, train_b.target)
(x_test, y_test) = (test_b.data, test_b.target)'''

# build, train, and validate model (Transformer is wrapper around transformers library)
import ktrain
from ktrain import text
MODEL_NAME = 'distilbert-base-uncased'
t = text.Transformer(MODEL_NAME, maxlen=500, class_names=Labels_names)
trn = t.preprocess_train(x_train, y_train)
val = t.preprocess_test(x_test, y_test)
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=6)
learner.fit_onecycle(5e-5, 1)



Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

preprocessing train...
language: en
train sequence lengths:
	mean : 1224
	95percentile : 1644
	99percentile : 1737


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 1235
	95percentile : 1629
	99percentile : 1738


Downloading:   0%|          | 0.00/363M [00:00<?, ?B/s]



begin training using onecycle policy with max lr of 5e-05...
1157/1157 [==============================] - 22851s 20s/step - loss: 2.0200 - accuracy: 0.3608 - val_loss: 1.7007 - val_accuracy: 0.4911


In [ ]:
model.save_pretrained('/content/drive/MyDrive/mymodel')

In [ ]:
# load model to generate embeddings
model.save_pretrained('/tmp/mymodel')
from transformers import *
import tensorflow as tf
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = TFAutoModel.from_pretrained('/content/drive/MyDrive/mymodel')
input_ids = tf.constant(tokenizer.encode("Hello, my dog is cute"))[None, :]  # Batch size 1
outputs = model(input_ids)
last_hidden_states = outputs[0]  # The last hidden-state is the first element of the output tuple
print(last_hidden_states.numpy().shape)

In [ ]:
y_train

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
'''NUM_WORDS = 50000
MAXLEN = 150
MODEL_NAME = 'bert-base-cased'  # replace this with model of choice
t = text.Transformer(MODEL_NAME, maxlen=500, class_names=Labels_names)
trn = t.preprocess_train(x_train, y_train)
val = t.preprocess_test(x_test, y_test)
model = t.get_classifier()'''

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

preprocessing train...
language: en
train sequence lengths:
	mean : 1224
	95percentile : 1644
	99percentile : 1737


Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 1235
	95percentile : 1629
	99percentile : 1738


Downloading:   0%|          | 0.00/527M [00:00<?, ?B/s]

In [ ]:
trn, val, preproc = text.texts_from_array(x_train=x_train, y_train=y_train,
                                          x_test=x_test, y_test=y_test,
                                          ngram_range=3,
                                          class_names=Labels_names,
                                          max_features=NUM_WORDS,
                                          maxlen=MAXLEN,
                                          preprocess_mode='distilbert')

preprocessing train...
language: en
train sequence lengths:
	mean : 1224
	95percentile : 1644
	99percentile : 1737


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 1235
	95percentile : 1629
	99percentile : 1738


task: text classification


In [ ]:
text.print_text_regression_models()

fasttext: a fastText-like model [http://arxiv.org/pdf/1607.01759.pdf]
linreg: linear text regression using a trainable Embedding layer
bigru: Bidirectional GRU with pretrained English word vectors [https://arxiv.org/abs/1712.09405]
standard_gru: simple 2-layer GRU with randomly initialized embeddings
bert: Bidirectional Encoder Representations from Transformers (BERT) - keras_bert implementation [https://arxiv.org/abs/1810.04805]
distilbert: distilled, smaller, and faster BERT from Hugging Face transformers [https://arxiv.org/abs/1910.01108]


In [ ]:
model = text.text_classifier('distilbert', trn, preproc=preproc, multilabel=True, verbose=1)

Is Multi-Label? True
maxlen is 150
done.


In [ ]:
model = text.models._build_transformer(num_classes=16, maxlen=15000)

Is Multi-Label? True
maxlen is 150
done.


In [ ]:
out

In [ ]:
no_of_unique_cat = np.max(LIWCs_train) + 1
embedding_size = min(np.ceil((no_of_unique_cat)/2), 50 )
embedding_size = int(embedding_size)
vocab =  int(no_of_unique_cat+1)
print(embedding_size)
extra_train = np.expand_dims(LIWCs_train, -1)
extra_test = np.expand_dims(LIWCs_test, -1)

# LIWCs module
extra_input = keras.layers.Input(shape=(1,))
extra_output = keras.layers.Embedding(vocab, embedding_size, input_length=1)(extra_input)
extra_output = keras.layers.Flatten()(extra_output)
extra_model = keras.Model(inputs=extra_input, outputs=extra_output)
extra_model.compile(loss='mse', optimizer='adam', metrics=['mae'])

50


In [ ]:
model.output_shape

AttributeError: ignored

In [ ]:
# Combine winery module with linreg model
#extra_in = keras.layers.Input(shape=(50,))(extra_model.output)
#bert_in = keras.layers.Input(shape=(512,))()
merged_out = keras.layers.concatenate([extra_model.output, model.output])
merged_out = keras.layers.Dropout(0.25)(merged_out)
merged_out = keras.layers.Dense(1000, activation='relu')(merged_out)
merged_out = keras.layers.Dropout(0.25)(merged_out)
merged_out = keras.layers.Dense(500, activation='relu')(merged_out)
merged_out = keras.layers.Dropout(0.5)(merged_out)
merged_out = keras.layers.Dense(1)(merged_out)
combined_model = keras.Model([extra_model.input] + [model.input], merged_out)
combined_model.compile(loss='mae',
                       optimizer='adam',
                      metrics=['mae'])

AttributeError: ignored

In [ ]:
#problème memoire
learner.lr_find(show_plot=True)

In [ ]:
learner.autofit(5e-3, 5)

In [ ]:
learner.view_top_losses(n=1, preproc=preproc)

##Multi-Label Text Classification

In [ ]:
NUM_WORDS = 50000
MAXLEN = 150
trn, val, preproc = text.texts_from_df(df,
                     'text',
                      label_columns = df.columns[1:].tolist(),
                      val_pct=0.1,
                      max_features=NUM_WORDS, maxlen=MAXLEN,
                      preprocess_mode= 'bert',
                      ngram_range=1)

['ENFJ', 'ENFP', 'ENTJ', 'ENTP', 'ESFJ', 'ESFP', 'ESTJ', 'ESTP', 'INFJ', 'INFP', 'INTJ', 'INTP', 'ISFJ', 'ISFP', 'ISTJ', 'ISTP']
      ENFJ  ENFP  ENTJ  ENTP  ESFJ  ESFP  ...  INTJ  INTP  ISFJ  ISFP  ISTJ  ISTP
1072     0     0     0     0     0     0  ...     0     0     0     0     0     0
8245     0     0     0     0     0     0  ...     0     0     0     0     0     0
4467     0     0     0     0     0     0  ...     0     0     0     0     0     0
6165     0     0     0     0     0     0  ...     1     0     0     0     0     0
2997     0     0     0     0     0     0  ...     0     0     0     0     0     0

[5 rows x 16 columns]
['ENFJ', 'ENFP', 'ENTJ', 'ENTP', 'ESFJ', 'ESFP', 'ESTJ', 'ESTP', 'INFJ', 'INFP', 'INTJ', 'INTP', 'ISFJ', 'ISFP', 'ISTJ', 'ISTP']
      ENFJ  ENFP  ENTJ  ENTP  ESFJ  ESFP  ...  INTJ  INTP  ISFJ  ISFP  ISTJ  ISTP
28       0     0     0     0     0     0  ...     0     0     0     0     0     0
4370     0     0     0     0     0     0  ...     1     0     0

Is Multi-Label? False
preprocessing test...
language: en


In [ ]:
text.print_text_classifiers()

fasttext: a fastText-like model [http://arxiv.org/pdf/1607.01759.pdf]
logreg: logistic regression using a trainable Embedding layer
nbsvm: NBSVM model [http://www.aclweb.org/anthology/P12-2018]
bigru: Bidirectional GRU with pretrained fasttext word vectors [https://fasttext.cc/docs/en/crawl-vectors.html]
standard_gru: simple 2-layer GRU with randomly initialized embeddings
bert: Bidirectional Encoder Representations from Transformers (BERT) from keras_bert [https://arxiv.org/abs/1810.04805]
distilbert: distilled, smaller, and faster BERT from Hugging Face transformers [https://arxiv.org/abs/1910.01108]


In [ ]:
model = text.text_classifier('distilbert', trn, preproc=preproc, multilabel=True)

Is Multi-Label? True
maxlen is 150
done.


In [ ]:
learner = ktrain.get_learner(model, train_data=trn, val_data=val)

In [ ]:
learner.lr_find()

simulating training for different learning rates... this may take a few moments...


/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 1/1024


In [ ]:
learner.lr_plot()

###Transformer

In [ ]:
df_train = df.sample(frac=0.85, random_state=42)
df_test = df.drop(df_train.index)

In [ ]:
x_train = df_train['text'].tolist()

In [ ]:
df_train.drop('text', axis=1, inplace=True)
y_train = df_train.values

In [ ]:
x_test = df_test['text'].tolist()

In [ ]:
df_test.drop('text', axis=1, inplace=True)
y_test = df_test.values

In [ ]:
x_test

In [ ]:
MODEL_NAME = 'bert-base-uncased'
t = text.Transformer(MODEL_NAME, maxlen=500,  
                     classes=df.columns[1:].tolist())
trn = t.preprocess_train(x_train, y_train)
val = t.preprocess_test(x_test, y_test)
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=6)
learner.fit_onecycle(3e-5, 1)

KeyboardInterrupt: ignored

###Multilabel auto classifier

In [ ]:
Df_data = DATA['Speech']

In [ ]:
Df_data

0        You're not battling traffickers or dealers, bu...
1        Back in the real world, we're talking about Me...
2        You want to make a difference, hit the users. ...
3        Mr. Wakefield, there's a situation that needs ...
4        Hello.  Hello. I'm sorry, Jonas.  I don't care...
                               ...                        
14900    This is a story of boy meets girl. The boy, To...
14901    I didn't know who else to call. Did she say wh...
14902    You did the right thing. Where is he? Thomas. ...
14903    Rachel? What are you doing here? I'm here to h...
14904    First, put down the plate. Now come here and s...
Name: Speech, Length: 14905, dtype: object

In [ ]:
%%time
tm = ktrain.text.get_topic_model(Df_data, n_topics=16, n_features=10000)

lang: en
preprocessing texts...
fitting model...
iteration: 1 of max_iter: 5
iteration: 2 of max_iter: 5
iteration: 3 of max_iter: 5
iteration: 4 of max_iter: 5
iteration: 5 of max_iter: 5
done.
CPU times: user 2min 12s, sys: 1min 26s, total: 3min 38s
Wall time: 2min 2s


In [ ]:
%%time
tm.build(Df_data, threshold=0.25)

done.
CPU times: user 21 s, sys: 10.4 s, total: 31.4 s
Wall time: 19.5 s


In [ ]:
tm.visualize_documents(doc_topics=tm.get_doctopics())

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
tm.print_topics(show_counts=True)

topic:3 | count:6139 | got right want yeah think gonna going okay come let
topic:10 | count:3333 | yes think want did going good tell say right time
topic:8 | count:2317 | looks door turns eyes takes hand face away head walks
topic:4 | count:1025 | man come father life love god time world dead let
topic:12 | count:858 | john people got need right case police time way shot
topic:0 | count:481 | frank car joe going okay old want really phone cindy
topic:9 | count:418 | sir president captain war ship men power army general matrix
topic:5 | count:48 | dad mom nick okay fox alex ted jason really sara
topic:1 | count:19 | kit ryan roger jay kay lily van margo storm blaze
topic:7 | count:11 | walter jacob jerry bobby walt celeste dicky bud caesar colin
topic:13 | count:10 | jack mary paul dave harry rose lisa sidney rev ruth
topic:11 | count:7 | steve ben elaine aaron johnny micky debbie yah brad robinson
topic:2 | count:7 | billy solomon ali rick schultz ginger fred alice bush buddy
topic:14